In [27]:
from stegano import lsb

In [29]:
# Hide a message in the image
secret_mgs = "@Terrorists"
stego_image = lsb.hide("Taliban.jpg", secret_mgs) # Original image

In [31]:
# Save the new image with hidden txt
stego_image.save("hidden_image.png") # Image with hidden text

print("Message hidden successfully!")

Message hidden successfully!


## Retrieve the Hidden Message

In [34]:
hidden_msg = lsb.reveal("hidden_image.png")
print("Hidden Message:", hidden_msg)

Hidden Message: @Terrorists


In [7]:
import cv2
import numpy as np
import random
import os
import speech_recognition as sr
from cryptography.fernet import Fernet
from dotenv import load_dotenv

# Load the AES key securely from .env file
load_dotenv()
AES_KEY = os.getenv("AES_KEY")

# Ensure the key is available and valid
if AES_KEY is None:
    raise ValueError("❌ AES_KEY not found! Make sure you have a valid .env file.")

try:
    cipher = Fernet(AES_KEY)  # No need to encode, it's already base64
except Exception as e:
    raise ValueError(f"❌ Invalid AES_KEY in .env! Error: {e}")

def decrypt_message(encrypted_message):
    """Decrypt message using AES-256 Fernet"""
    try:
        decrypted_data = cipher.decrypt(encrypted_message.encode())
        return decrypted_data.decode()
    except Exception as e:
        print(f"❌ Decryption failed: {e}")
        return None

def extract_message(image_path, secret_key):
    """Extracts the hidden message using randomized LSB steganography"""
    image = cv2.imread(image_path)
    if image is None:
        print("❌ Error: Image not found!")
        return None

    h, w, _ = image.shape
    total_pixels = h * w * 3
    random.seed(secret_key)
    positions = random.sample(range(total_pixels), total_pixels)

    binary_message = ""
    for pos in positions:
        x = (pos // 3) % w
        y = (pos // (3 * w))
        channel = pos % 3
        binary_message += str(image[y, x, channel] & 1)

        if binary_message.endswith("1111111111111110"):  # End marker
            try:
                encrypted_message = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message)-16, 8))
                decrypted_message = decrypt_message(encrypted_message)
                return decrypted_message
            except Exception as e:
                print(f"❌ Error extracting message: {e}")
                return None

    print("❌ No hidden message found!")
    return None

def authenticate_voice(passphrase):
    """Authenticate user using voice recognition"""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎙 Speak the secret passphrase...")
        try:
            audio = recognizer.listen(source, timeout=5)
            spoken_text = recognizer.recognize_google(audio)
            return spoken_text.lower() == passphrase.lower()
        except:
            return False

# 🔹 Multi-Level Authentication 🔹
SECRET_PASSPHRASE = "follow me"  # Set a highly secure passphrase

if authenticate_voice(SECRET_PASSPHRASE):
    print("✅ Voice Authentication Successful")
    extracted_message = extract_message("encoded_secure.png", secret_key=12345)
    if extracted_message:
        print(f"🔓 Decrypted Message: {extracted_message}")
    else:
        print("❌ No hidden message found.")
else:
    print("❌ Voice Authentication Failed!")


🎙 Speak the secret passphrase...
✅ Voice Authentication Successful
❌ Decryption failed: 
❌ No hidden message found.


In [27]:
from cryptography.fernet import Fernet

# Generate a new AES-256 key
new_key = Fernet.generate_key()

# Save the key securely
print(f"Your secure AES-256 key: {new_key.decode()}")


Your secure AES-256 key: jIUB0NqhvTsYHu3N1iiEe-pDPQ8_qFlYsp6BDnPjKnY=


In [5]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print(key.decode())  # Copy this key and store it in your .env file as AES_KEY


PwDqJjp3p30y_qeNPTaJIlqJ7f_3LEn3e7NEBl0CpkI=


In [3]:
import hashlib

auth_code = "follow me"  # Change this to your secret passphrase
hashed_auth = hashlib.sha256(auth_code.encode()).hexdigest()
print("Hashed Authentication Code:", hashed_auth)


Hashed Authentication Code: f419ac24c4a9bd376a9f16bfc9c840267e293c04bfbadffc3acaaa8ef81ae2d2


In [8]:
import cv2
import numpy as np
import random
import hashlib
from Crypto.Cipher import AES
import base64

# AES Encryption/Decryption Functions
def decrypt_message(encrypted_message, key):
    """Decrypts AES-256 encrypted message"""
    key = hashlib.sha256(key.encode()).digest()  # Generate strong key
    encrypted_message = base64.b64decode(encrypted_message)
    iv = encrypted_message[:16]  # Extract IV
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(encrypted_message[16:]).decode('utf-8', errors="ignore")
    return decrypted.rstrip()  # Remove padding

def extract_message(image_path, secret_key, auth_code):
    """Extracts encrypted message using randomized LSB steganography"""
    
    # Authentication Check
    if hashlib.sha256(auth_code.encode()).hexdigest() != "f419ac24c4a9bd376a9f16bfc9c840267e293c04bfbadffc3acaaa8ef81ae2d2":  # Replace with pre-set hash
        print("❌ Unauthorized Access! Data will self-destruct.")
        return None  # Exit without revealing message
    
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found!")
        return None
    
    h, w, _ = image.shape
    total_pixels = h * w * 3
    random.seed(secret_key)  # Ensure same randomness as embedding
    positions = random.sample(range(total_pixels), total_pixels)
    
    binary_message = ""
    for pos in positions:
        x = (pos // 3) % w
        y = (pos // (3 * w))
        channel = pos % 3
        binary_message += str(image[y, x, channel] & 1)
        
        if binary_message.endswith("1111111111111110"):
            binary_message = binary_message[:-16]  # Remove end marker
            encrypted_text = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
            decrypted_message = decrypt_message(encrypted_text, secret_key)
            
            # Tamper Detection
            original_hash = hashlib.sha256(decrypted_message.encode()).hexdigest()
            if original_hash != "8b12e6a4d...":  # Replace with correct hash
                print("⚠️ WARNING: Data has been altered! Extraction blocked.")
                return None
            
            print("✅ Secure Message Extracted:", decrypted_message)
            return decrypted_message
    
    print("No valid message found!")
    return None

# Example Usage
extract_message("encoded_secure.png", secret_key="BlackOps123", auth_code="RAW_SECRET_AUTH")


❌ Unauthorized Access! Data will self-destruct.


In [22]:
import cv2
import numpy as np
import random
import hashlib
from Crypto.Cipher import AES
import base64

# AES Decryption Function
def decrypt_message(encrypted_message, key):
    """Decrypts AES-256 encrypted message with proper padding handling"""
    try:
        key = hashlib.sha256(key.encode()).digest()  # Generate strong key
        encrypted_message = base64.b64decode(encrypted_message.encode())  # Ensure proper encoding
        iv = encrypted_message[:16]  # Extract IV
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted = cipher.decrypt(encrypted_message[16:])

        # Remove PKCS7 Padding
        pad = decrypted[-1]  # Last byte contains padding length
        if isinstance(pad, int) and 1 <= pad <= 16:  # Valid padding check
            decrypted = decrypted[:-pad]  # Remove padding

        return decrypted.decode('utf-8', errors="ignore")
    except Exception as e:
        print("❌ Decryption Error:", str(e))
        return None

# Message Extraction from Image
def extract_message(image_path, secret_key, auth_code):
    """Extracts encrypted message using randomized LSB steganography"""
    
    # Authentication Check
    stored_auth_hash = "f419ac24c4a9bd376a9f16bfc9c840267e293c04bfbadffc3acaaa8ef81ae2d2"  # Pre-set hash
    if hashlib.sha256(auth_code.encode()).hexdigest() != stored_auth_hash:
        print("❌ Unauthorized Access! Data will self-destruct.")
        return None  
    
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found!")
        return None
    
    h, w, _ = image.shape
    total_pixels = h * w * 3
    random.seed(secret_key)  # Ensure same randomness as embedding
    positions = random.sample(range(total_pixels), total_pixels)
    
    binary_message = ""
    for pos in positions:
        x = (pos // 3) % w
        y = (pos // (3 * w))
        channel = pos % 3
        binary_message += str(image[y, x, channel] & 1)
        
        if binary_message.endswith("1111111111111110"):  # Stop marker
            binary_message = binary_message[:-16]  # Remove marker
            try:
                encrypted_text = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
                encrypted_text = encrypted_text.strip()  # Ensure no extra spaces
                
                decrypted_message = decrypt_message(encrypted_text, secret_key)
                if decrypted_message is None:
                    return None
                
                # Tamper Detection
                expected_hash = "8b12e6a4d..."  # Replace with correct hash
                if hashlib.sha256(decrypted_message.encode()).hexdigest() != expected_hash:
                    print("⚠️ WARNING: Data has been altered! Extraction blocked.")
                    return None
                
                print("✅ Secure Message Extracted:", decrypted_message)
                return decrypted_message
            except Exception as e:
                print("❌ Decryption Error:", str(e))
                return None
    
    print("No valid message found!")
    return None

# Example Usage
extract_message("encoded_secure.png", secret_key="BlackOps123", auth_code="follow me")


❌ Decryption Error: Data must be padded to 16 byte boundary in CBC mode
